In [1]:
import time

# get the start time
st = time.time()

In [2]:
import numpy as np
import math

# set the random seed
np.random.seed(42)

import strawberryfields as sf
from strawberryfields.ops import *

ham_simulation = sf.Program(2)

# set the Hamiltonian parameters
J = 2           # hopping transition
U = 4        # on-site interaction
k = 100         # Lie product decomposition terms
t = 0.5       # timestep
theta = -J*t/k
r = -U*t/(2*k)

with ham_simulation.context as q:
    # prepare the initial state
    Fock(2) | q[0]

    # Two node tight-binding
    # Hamiltonian simulation

    for i in range(k):
        BSgate(theta, np.pi/4) | (q[0], q[1])
        Kgate(r)  | q[0]
        Rgate(-r) | q[1]
        Kgate(r)  | q[0]
        Rgate(-r) | q[1]

In [3]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})


In [4]:
results = eng.run(ham_simulation)


In [5]:
# wait for 5 seconds
time.sleep(5)

In [6]:
state = results.state
print("P(|0, 2>) = ", state.fock_prob([0, 2]))
print("P(|1, 1>) = ", state.fock_prob([1, 1]))
print("P(|2, 0>) = ", state.fock_prob([2, 0]))


P(|0, 2>) =  0.006123830455131159
P(|1, 1>) =  0.05971765105003921
P(|2, 0>) =  0.9341585184948159


In [7]:
result = [state.fock_prob([0, 2]), state.fock_prob([1, 1]), state.fock_prob([2, 0])]
print(np.sum(result))

0.9999999999999862


In [8]:
from scipy.linalg import expm

H = J*np.sqrt(2)*np.array([[0,1,0],[1,0,1],[0,1,0]]) + U*np.diag([1,0,1])
init_state = np.array([0,0,1])

theoretical_result = np.abs(np.dot(expm(-1j*t*H), init_state))**2
print(theoretical_result)

[0.3949285  0.24758968 0.35748182]


In [9]:
print(np.all(np.abs(theoretical_result - result) < 1e-2))


False


In [10]:
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 9.38537883758545 seconds


In [11]:
fidelity = math.sqrt(results.state.fidelity_coherent([0.7+1.2j, 0.7+1.2j]))
print(fidelity)

0.2053677945343228
